# Chapter7. 집계 연산

집계를 수행하기 위해서는 키나 그룹을 지정하고 하나 이상의 컬럼을 변환하는 방법을 지정하는 집계 함수 사용. 

* 집계 함수 : 여러 입력 값이 주어지면 그룹별로 결과 생성
* 수치형 데이터 요약에 집계 사용 - 합산, 곱셈 또는 카운팅 등
* 배열, 리스트 또는 맵 같은 복합 데이터 타입을 사용해 집계 수행 가능

< 그룹화 데이터 타입 생성 방법 >

* select 구문에서 집계를 수행해 DataFrame의 전체 데이터 요약
* group by는 하나 이상의 키를 지정할 수 있으며 값을 가진 컬럼을 변환하기 위해 다른 집계 함수 사용 가능
* window는 하나 이상의 키를 지정할 수 있으며 값을 가진 컬럼을 변환하기 위해 다른 집계 함수 사용 가능 but 함수의 입력으로 사용할 로우는 현재 로우와 연관성이 있어야 함
* grouping set은 서로 다른 레벨의 값을 집계할 때 사용 (SQL, DataFrame의 롤업, 큐브 사용 가능)
* rollup은 하나 이상의 키를 지정 가능. 컬럼을 변환하는데 다른 집계 함수를 사용해 계층적으로 요약된 값을 구할 수 있음.
* cube는 하나 이상의 키를 지정할 수 있으며 값을 가진 컬럼을 변환하기 위해 다른 집계 함수를 사용 가능. (큐브는 모든 컬럼 조합에 대한 요약 값 계산)

지정된 집계 함수에 따라 그룹화된 결과는 RelationalGroupedDataset 반환


코드) 구매 이력 데이터를 사용해 파티션을 훨씬 적은 수로 분할할 수 있도록 리파티셔닝하고 빠르게 접근할 수 있도록 캐싱

(파티션 수를 줄이는 이유 : 적은 양의 데이터를 가진 수많은 파일이 존재하기 때문)

In [0]:
df = spark.read.format("csv")\
  .option("header", "true")\
  .option("inferSchema", "true")\
  .load("dbfs:/FileStore/shared_uploads/hyjeong0815@gmail.com/*.csv")\
  .coalesce(5)

df.cache()
df.createOrReplaceTempView("dfTable")

## 7.1 집계 함수

DataFrame의 .stat 속성을 이용하는 특별한 경우를 제외하면 함수를 사용한다.

### 7.1.1 count

count 함수는 액션이 아닌 트랜스포메이션으로 동작. 

< count 함수 사용 방식 >
* count 함수에 특정 컬럼을 지정
* count( * )나 count(1)을 사용하는 방식

count 함수를 통해 다음과 같이 전체 로우 수를 카운트할 수 있음.

In [0]:
from pyspark.sql.functions import count

df.select(count("StockCode")).show()

+----------------+
count(StockCode)|
+----------------+
 1098096|
+----------------+

### 7.1.2 countDistinct

전체 레코드 수가 아닌 고유 레코드 수를 구해야 한다면 개별 컬럼을 처리에 더 적합한 countDistinct 함수를 사용.

In [0]:
from pyspark.sql.functions import countDistinct

df.select(countDistinct("StockCode")).show()

+-------------------------+
count(DISTINCT StockCode)|
+-------------------------+
 4195|
+-------------------------+

### 7.1.3 approx_count_distinct

대규모 데이터셋을 다루다 보면 정확한 고유 개수가 무의미한 경우도 있는데, 어느 정도 수준의 정확도를 가지는 근사치만으로도 유의미하다면 approx_count_distinct 함수를 사용해 근사치 계산 가능

In [0]:
from pyspark.sql.functions import approx_count_distinct

df.select(approx_count_distinct("StockCode", 0.1)).show()

+--------------------------------+
approx_count_distinct(StockCode)|
+--------------------------------+
 3465|
+--------------------------------+

approx_count_distinct 함수는 최대 추정 오류율이라는 파라미터를 사용한다. 위에서 지정한 0.1이 여기에 해당.

위에서는 큰 오류율을 설정했기 때문에 기대치에서 크게 벗어나는 결과를 얻게 되지만 countDistinct 함수보다 더 빠르게 결과를 반환해 대규모 데이터셋을 사용할 때 더 좋아짐.

### 7.1.4 first와 last

fist와 last는 DataFrame의 첫 번째 값이나 마지막 값을 얻을 때 사용하는 함수로, DataFrame의 값이 아닌 로우를 기반으로 동작.

In [0]:
from pyspark.sql.functions import first, last

df.select(first("StockCode"), last("StockCode")).show()

+----------------+---------------+
first(StockCode)|last(StockCode)|
+----------------+---------------+
 85123A| United States|
+----------------+---------------+

### 7.1.5 min과 max

min과 max 함수는 DataFrame에서 최솟값과 최댓값을 추출.

In [0]:
from pyspark.sql.functions import min, max

df.select(min("Quantity"), max("Quantity")).show()

+-------------+-------------+
min(Quantity)|max(Quantity)|
+-------------+-------------+
 -80995| 80995|
+-------------+-------------+

### 7.1.6 sum

sum 함수는 DataFrame에서 특정 컬럼의 모든 값을 합산하기 위해 사용.

In [0]:
from pyspark.sql.functions import sum

df.select(sum("Quantity")).show()

+-------------+
sum(Quantity)|
+-------------+
 10453381|
+-------------+

### 7.1.7 sumDistinct

특정 컬럼의 모든 값을 합산하는 방법 외에도 sumDistinct 함수를 사용해 고윳값 합산 가능.

In [0]:
from pyspark.sql.functions import sumDistinct

df.select(sumDistinct("Quantity")).show()

+----------------------+
sum(DISTINCT Quantity)|
+----------------------+
 29310|
+----------------------+

### 7.1.8 avg

avg 함수나 mean 함수를 사용해 평균값을 구할 수 있다. 

코드) 집계된 컬럼을 재활용하기 위해 alias 메서드 사용

In [0]:
from pyspark.sql.functions import sum, count, avg, expr

df.select(
  count("Quantity").alias("total_transactions"),
  sum("Quantity").alias("total_purchases"),
  avg("Quantity").alias("avg_purchases"),
  expr("mean(Quantity)").alias("mean_purchases"))\
  .selectExpr(
    "total_purchases/total_transactions",
    "avg_purchases",
    "mean_purchases").show()

+--------------------------------------+----------------+----------------+
(total_purchases / total_transactions)| avg_purchases| mean_purchases|
+--------------------------------------+----------------+----------------+
 9.52177093201195|9.52177093201195|9.52177093201195|
+--------------------------------------+----------------+----------------+

### 7.1.9 분산과 표준편차

분산과 표준편차는 평균 주변에 데이터가 분포된 정도를 측정하는 방법으로, 분산은 평균과의 차이를 제곱한 결과의 평균이고, 표준편차는 분산의 제곱근이다. 

스파크는 variance 함수나 stddev 함수를 사용해 표본표준분산과 표본표준편차 공식을 이용. 

(모표준분산이나ㅣ 모표준편차 방식을 사용하려면 var_pop 함수나 stddev_pop 함수 사용)

In [0]:
from pyspark.sql.functions import var_pop, stddev_pop
from pyspark.sql.functions import var_samp, stddev_samp

df.select(var_pop("Quantity"), var_samp("Quantity"),
          stddev_pop("Quantity"), stddev_samp("Quantity")).show()

+------------------+------------------+--------------------+---------------------+
 var_pop(Quantity)|var_samp(Quantity)|stddev_pop(Quantity)|stddev_samp(Quantity)|
+------------------+------------------+--------------------+---------------------+
47052.105131579396| 47052.14799041856| 216.91497212405463| 216.9150709158277|
+------------------+------------------+--------------------+---------------------+

### 7.1.10 비대도와 첨도

비대칭도와 첨도는 데이터의 변곡점을 측정하는 방법으로 비대칭도는 데이터 평균의 비대칭 정도를 측정하고, 첨도는 데이터 끝 부분을 측정한다. 

비대칭도와 첨도는 확률변수의 확률분포로 데이터를 모델링할 때 특히 중요한다.

In [0]:
from pyspark.sql.functions import skewness, kurtosis

df.select(skewness("Quantity"), kurtosis("Quantity")).show()

+--------------------+------------------+
 skewness(Quantity)|kurtosis(Quantity)|
+--------------------+------------------+
-0.33462002416552633| 120804.4086848774|
+--------------------+------------------+

### 7.1.11 공분산과 상관관계

cov와 corr 함수를 사용해 두 컬럼값 사이의 공분산과 상관관계를 계산할 수 있는데 공분산은 데이터 입력값에 따라 다른 범위를 가지지만 상관관계는 피어슨 상관계수를 측정하므로 -1과 1 시아의 값을 가진다.

var 함수처럼 표본공분산 방식이나 모공분산 방식으로 공분산을 계산할 수 있으므로 사용하고자 하는 방식을 명확하게 지정해야 함. 

(상관관계는 모집단이나 표본에 대한 계산 개념이 없음.)

In [0]:
from pyspark.sql.functions import corr, covar_pop, covar_samp

df.select(corr("InvoiceNo", "Quantity"), covar_samp("InvoiceNo", "Quantity"),
          covar_pop("InvoiceNo", "Quantity")).show()

+-------------------------+-------------------------------+------------------------------+
corr(InvoiceNo, Quantity)|covar_samp(InvoiceNo, Quantity)|covar_pop(InvoiceNo, Quantity)|
+-------------------------+-------------------------------+------------------------------+
 7.756263509409191E-4| 1672.2220090155577| 1672.220459383399|
+-------------------------+-------------------------------+------------------------------+

### 7.1.12 복합 데이터 타입의 집계

스파크는 수식을 이용한 집계뿐만 아니라 복합 데이터 타입을 사용해 특정 컬럼의 값을 리스트로 수집하거나 셋 데이터 타입으로 고윳값만 수집하는 등의 집계도 수행 가능. 

수집된 데이터는 처리 파이프라인에서 다양한 프로그래밍 방식으로 다루거나 사용자 정의 함수를 사용해 전체 데이터에 접근 가능.

In [0]:
from pyspark.sql.functions import collect_set, collect_list

df.agg(collect_set("Country"), collect_list("Country")).show()

+--------------------+---------------------+
collect_set(Country)|collect_list(Country)|
+--------------------+---------------------+
[Portugal, Italy,...| [United Kingdom, ...|
+--------------------+---------------------+

## 7.2 그룹화

DataFrame 수준의 집계보다 그룹 기반의 집계를 수행하는 경우가 더 많다. 

데이터 그룹 기반의 집계는 단일 컬럼의 데이터를 그룹화하고 해당 그룹의 다른 여러 컬럼을 사용해 계산하기 위해 카테고리형 데이터 사용. 

그룹화 작업은 1) 하나 이상의 컬럼을 그룹화 2) 집계 연산 수행의 두 단계로 이루어진다.

첫 번째 단계에서 RelationalGroupedDataset이 반환되고, 두 번째 단계에서 DataFrame이 반환.

In [0]:
df.groupBy("InvoiceNo", "CustomerID").count().show()

+---------+----------+-----+
InvoiceNo|CustomerID|count|
+---------+----------+-----+
 536738| 15827.0| 21|
 536860| 16905.0| 30|
 C536979| 17841.0| 6|
 C538084| 12649.0| 4|
 C538085| 16350.0| 6|
 538660| 15808.0| 84|
 C540177| 16316.0| 2|
 540753| 14217.0| 8|
 541269| 12626.0| 106|
 541789| 14088.0| 186|
 542093| 17677.0| 32|
 542208| 16422.0| 20|
 543998| 18093.0| 8|
 544161| 16875.0| 44|
 544604| 13576.0| 4|
 544919| 13725.0| 20|
 544996| 12474.0| 26|
 545016| 16473.0| 10|
 545398| 13722.0| 92|
 545700| 17811.0| 32|
+---------+----------+-----+
only showing top 20 rows

### 7.2.1 표현식을 사용한 그룹화

카운팅은 메서드와 함수로 사용할 수 있는데 count 함수를 사용하는 것이 더 좋다. 또한 count 함수를 select 구문에 표현식으로 지정하는 것보다 agg 메서드를 사용하는 것이 좋다. 

그 이유는 agg 메서드는 여러 집계 처리를 한 번에 지정할 수 있고 집계에 표현식을 사용할 수 있기 때문. 또한 트랜스포메이션이 완료된 컬럼에 alias 메서드 사용 가능.

In [0]:
from pyspark.sql.functions import count

df.groupBy("InvoiceNo").agg(
  count("Quantity").alias("quan"),
  expr("count(Quantity)")).show()

+---------+----+---------------+
InvoiceNo|quan|count(Quantity)|
+---------+----+---------------+
 536596| 18| 18|
 536938| 42| 42|
 537252| 3| 3|
 537691| 40| 40|
 538041| 2| 2|
 538184| 52| 52|
 538517| 106| 106|
 538879| 38| 38|
 539275| 12| 12|
 539630| 24| 24|
 540499| 48| 48|
 540540| 44| 44|
 C540850| 2| 2|
 540976| 96| 96|
 541432| 8| 8|
 541518| 202| 202|
 541783| 70| 70|
 542026| 18| 18|
 542375| 12| 12|
 C542604| 16| 16|
+---------+----+---------------+
only showing top 20 rows

### 7.2.2 맵을 이용한 그룹화

컬럼을 키로, 수행할 집계 함수의 문자열을 값으로 하는 맵 타입을 사용해 트랜스포메이션 정의 가능.

또한 수행할 집계 함수를 한 줄로 작성하면 여러 컬럼명 재사용 가능.

In [0]:
df.groupBy("InvoiceNo").agg(expr("avg(Quantity)"), expr("stddev_pop(Quantity)"))\
  .show()

+---------+------------------+--------------------+
InvoiceNo| avg(Quantity)|stddev_pop(Quantity)|
+---------+------------------+--------------------+
 536596| 1.5| 1.1180339887498947|
 536938|33.142857142857146| 20.698023172885524|
 537252| 31.0| 0.0|
 537691| 8.15| 5.597097462078001|
 538041| 30.0| 0.0|
 538184|12.076923076923077| 8.142590198943392|
 538517|3.0377358490566038| 2.3946659604837897|
 538879|21.157894736842106| 11.811070444356483|
 539275| 26.0| 12.806248474865697|
 539630|20.333333333333332| 10.225241100118645|
 540499| 3.75| 2.6653642652865788|
 540540|2.1363636363636362| 1.0572457590557278|
 C540850| -1.0| 0.0|
 540976|10.520833333333334| 6.496760677872902|
 541432| 12.25| 10.825317547305483|
 541518| 23.10891089108911| 20.550782784878713|
 541783|11.314285714285715| 8.467657556242811|
 542026| 7.666666666666667| 4.853406592853679|
 542375| 8.0| 3.4641016151377544|
 C542604| -8.0| 15.173990905493518|
+---------+------------------+--------------------+
only showing top 20 rows

## 7.3 윈도우 함수

윈도우 함수는 특정 윈도우(window)를 대상으로 고유의 집계 연산을 수행한다. 데이터의 윈도우는 현재 데이터에 대한 참로를 사용해 정의. 

윈도우 명세는 함수에 전달될 로우를 결정. 

group-by 함수를 사용하면 모든 로우 레코드가 단일 그룹으로만 이동. but 윈도우 함수는 프레임(로우 그룹 기반의 테이블)에 입력되는 모든 로우에 대한 결과값을 계산. 

각 로우는 하나 이상의 프레임에 할당 가능.

< 스파크가 지원하는 세 가지 종류의 윈도우 함수 >
* 랭크 함수(ranking function)
* 분석 함수(analytic function)
* 집계 함수(aggregate function)

코드) 주문 일자(InvoiceDate) 컬럼을 변환해 date 컬럼을 만드는데, 시간 정보를 제외한 날짜 정보만 가짐.

In [0]:
from pyspark.sql.functions import col, to_date

dfWithDate = df.withColumn("date", to_date(col("InvoiceDate"), "MM/d/yyyy H:mm"))
dfWithDate.createOrReplaceTempView("dfWithDate")

윈도우 함수를 정의하기 위해 첫 번째 단계로 윈도우 명세 생성.

* partitionBy 메서드 : 그룹을 어떻게 나눌지 결정하는 것과 유사한 개념
* orderBy 메서드 : 파티션의 정렬 방ㅅ힉을 정의
* 프레임 명세(rowsBetween 구문) : 입력된 로우의 참조를 기반으로 프레임에 로우가 포함될 수 있는지 결정

코드) 첫 로우부터 현재 로우까지 확인

In [0]:
from pyspark.sql.window import Window
from pyspark.sql.functions import desc

windowSpec = Window\
  .partitionBy("CustomerId", "date")\
  .orderBy(desc("Quantity"))\
  .rowsBetween(Window.unboundedPreceding, Window.currentRow)

코드) 시간대별 최대 구매 개수 구하기 

코드를 구현하기 위해서는 집계 함수에 컬럼명이나 표현식을 전달. 그리고 이 함수에 적용할 데이터 프레임이 정의된 윈도우 명세도 함께 사용.

In [0]:
from pyspark.sql.functions import max

maxPurchaseQuantity = max(col("Quantity")).over(windowSpec)

위의 예제는 컬럼이나 표현식을 반환하므로 DataFrame의 select 구문에서 사용할 수 있다. 

코드) 구매량 순위를 생성 - dense_rank 함수를 사용해 모든 고객에 대해 최대 구매 수량을 가진 날짜가 언제인지 알아보기. 

(동일한 값이 나오거나 중복 로우가 발생해 순위가 비어 있을 수 있으므로 rank 함수 대신 dense_rank 함수 사용)

In [0]:
from pyspark.sql.functions import dense_rank, rank

purchaseDenseRank = dense_rank().over(windowSpec)
purchaseRank = rank().over(windowSpec)

코드) select 메서드를 사용해 계산된 윈도우값 확인

In [0]:
from pyspark.sql.functions import col

dfWithDate.where("CustomerId IS NOT NULL").orderBy("CustomerId")\
  .select(
    col("CustomerId"),
    col("date"),
    col("Quantity"),
    purchaseRank.alias("quantityRank"),
    purchaseDenseRank.alias("quantityDenseRank"),
    maxPurchaseQuantity.alias("maxPurchaseQuantity")).show()

--------------------------------------------------------------------------- 
 Py4JJavaError Traceback (most recent call last)
 <command-1049518272702046> in <module> 
 1 from pyspark . sql . functions import col
 2 
 ----> 3 dfWithDate . where ( "CustomerId IS NOT NULL" ) . orderBy ( "CustomerId" ) \ 
 4 .select(
 5 col ( "CustomerId" ) , 

 /databricks/spark/python/pyspark/sql/dataframe.py in show (self, n, truncate, vertical) 
 488 """
 489 if isinstance ( truncate , bool ) and truncate : 
 --> 490 print ( self . _jdf . showString ( n , 20 , vertical ) ) 
 491 else : 
 492 print ( self . _jdf . showString ( n , int ( truncate ) , vertical ) ) 

 /databricks/spark/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1302 
 1303 answer = self . gateway_client . send_command ( command ) 
 -> 1304 return_value = get_return_value(
 1305 answer, self.gateway_client, self.target_id, self.name)
 1306 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 115 def deco ( * a , ** kw ) : 
 116 try : 
 --> 117 return f ( * a , ** kw ) 
 118 except py4j . protocol . Py4JJavaError as e : 
 119 converted = convert_exception ( e . java_exception ) 

 /databricks/spark/python/lib/py4j-0.10.9-src.zip/py4j/protocol.py in get_return_value (answer, gateway_client, target_id, name) 
 324 value = OUTPUT_CONVERTER [ type ] ( answer [ 2 : ] , gateway_client ) 
 325 if answer [ 1 ] == REFERENCE_TYPE : 
 --> 326 raise Py4JJavaError(
 327 "An error occurred while calling {0}{1}{2}.\n" . 
 328 format(target_id, ".", name), value)

 Py4JJavaError : An error occurred while calling o1545.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 1 in stage 61.0 failed 1 times, most recent failure: Lost task 1.0 in stage 61.0 (TID 125) (ip-10-172-246-109.us-west-2.compute.internal executor driver): org.apache.spark.SparkUpgradeException: You may get a different result due to the upgrading of Spark 3.0: Fail to parse '5/13/2011 11:45' in the new parser. You can set spark.sql.legacy.timeParserPolicy to LEGACY to restore the behavior before Spark 3.0, or set to CORRECTED and treat it as an invalid datetime string.
	at org.apache.spark.sql.catalyst.util.DateTimeFormatterHelper$$anonfun$checkParsedDiff$1.applyOrElse(DateTimeFormatterHelper.scala:150)
	at org.apache.spark.sql.catalyst.util.DateTimeFormatterHelper$$anonfun$checkParsedDiff$1.applyOrElse(DateTimeFormatterHelper.scala:141)
	at scala.runtime.AbstractPartialFunction.apply(AbstractPartialFunction.scala:38)
	at org.apache.spark.sql.catalyst.util.Iso8601TimestampFormatter.$anonfun$parse$1(TimestampFormatter.scala:86)
	at scala.runtime.java8.JFunction0$mcJ$sp.apply(JFunction0$mcJ$sp.java:23)
	at scala.Option.getOrElse(Option.scala:189)
	at org.apache.spark.sql.catalyst.util.Iso8601TimestampFormatter.parse(TimestampFormatter.scala:77)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.java:43)
	at org.apache.spark.sql.execution.WholeStageCodegenExec$$anon$1.hasNext(WholeStageCodegenExec.scala:757)
	at scala.collection.Iterator$$anon$10.hasNext(Iterator.scala:458)
	at scala.collection.Iterator$$anon$10.hasNext(Iterator.scala:458)
	at org.apache.spark.util.random.SamplingUtils$.reservoirSampleAndCount(SamplingUtils.scala:41)
	at org.apache.spark.RangePartitioner$.$anonfun$sketch$1(Partitioner.scala:306)
	at org.apache.spark.RangePartitioner$.$anonfun$sketch$1$adapted(Partitioner.scala:304)
	at org.apache.spark.rdd.RDD.$anonfun$mapPartitionsWithIndex$2(RDD.scala:920)
	at org.apache.spark.rdd.RDD.$anonfun$mapPartitionsWithIndex$2$adapted(RDD.scala:920)
	at org.apache.spark.rdd.MapPartitionsRDD.compute(MapPartitionsRDD.scala:60)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoint(RDD.scala:380)
	at org.apache.spark.rdd.RDD.iterator(RDD.scala:344)
	at org.apache.spark.scheduler.ResultTask.$an

## 7.4 그룹화 셋

그룹화 셋은 여러 집계를 결합하는 저수준 기능. 그룹화 셋을 이용하면 group-by 구문에서 원하는 형태로 집곗 생성 가능. 

코드) 재고 코드(StockCode)와 고객(CustomerId)별 총 수량을 얻기 위해 SQL 표현식 사용

In [0]:
dfNoNull = dfWithDate.drop()
dfNoNull.createOrReplaceTempView("dfNoNull")

그룹화 셋을 사용해 동일한 작업 수행 가능 but 그룹화 셋(GROUPING SETS 구문)은 SQL에서만 사용 가능.

DataFrame에서 동일한 연산을 수행하려면 rollup 메서드와 cube 메서드 사용.

### 7.4.1 롤업

롤업은 group-by 스타일의 다양한 연산을 수행할 수 있는 다차원 집계 기능. 

코드) 시간(신규 Date 컬럼)과 공간(Country 컬럼)을 축으로 하는 롤업 생성 - 롤업의 결과로 생성된 DataFrame은 모든 날짜의 총합, 날짜별 총합, 날짜별 국가별 총합 포함.

In [0]:
rolledUpDF = dfNoNull.rollup("Date", "Country").agg(sum("Quantity"))\
  .selectExpr("Date", "Country", "`sum(Quantity)` as total_quantity")\
  .orderBy("Date")
rolledUpDF.show()

--------------------------------------------------------------------------- 
 Py4JJavaError Traceback (most recent call last)
 <command-1049518272702051> in <module> 
 2 . selectExpr ( "Date" , "Country" , "`sum(Quantity)` as total_quantity" ) \ 
 3 . orderBy ( "Date" ) 
 ----> 4 rolledUpDF . show ( ) 

 /databricks/spark/python/pyspark/sql/dataframe.py in show (self, n, truncate, vertical) 
 488 """
 489 if isinstance ( truncate , bool ) and truncate : 
 --> 490 print ( self . _jdf . showString ( n , 20 , vertical ) ) 
 491 else : 
 492 print ( self . _jdf . showString ( n , int ( truncate ) , vertical ) ) 

 /databricks/spark/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1302 
 1303 answer = self . gateway_client . send_command ( command ) 
 -> 1304 return_value = get_return_value(
 1305 answer, self.gateway_client, self.target_id, self.name)
 1306 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 115 def deco ( * a , ** kw ) : 
 116 try : 
 --> 117 return f ( * a , ** kw ) 
 118 except py4j . protocol . Py4JJavaError as e : 
 119 converted = convert_exception ( e . java_exception ) 

 /databricks/spark/python/lib/py4j-0.10.9-src.zip/py4j/protocol.py in get_return_value (answer, gateway_client, target_id, name) 
 324 value = OUTPUT_CONVERTER [ type ] ( answer [ 2 : ] , gateway_client ) 
 325 if answer [ 1 ] == REFERENCE_TYPE : 
 --> 326 raise Py4JJavaError(
 327 "An error occurred while calling {0}{1}{2}.\n" . 
 328 format(target_id, ".", name), value)

 Py4JJavaError : An error occurred while calling o1654.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 1 in stage 63.0 failed 1 times, most recent failure: Lost task 1.0 in stage 63.0 (TID 135) (ip-10-172-246-109.us-west-2.compute.internal executor driver): org.apache.spark.SparkUpgradeException: You may get a different result due to the upgrading of Spark 3.0: Fail to parse '5/13/2011 11:45' in the new parser. You can set spark.sql.legacy.timeParserPolicy to LEGACY to restore the behavior before Spark 3.0, or set to CORRECTED and treat it as an invalid datetime string.
	at org.apache.spark.sql.catalyst.util.DateTimeFormatterHelper$$anonfun$checkParsedDiff$1.applyOrElse(DateTimeFormatterHelper.scala:150)
	at org.apache.spark.sql.catalyst.util.DateTimeFormatterHelper$$anonfun$checkParsedDiff$1.applyOrElse(DateTimeFormatterHelper.scala:141)
	at scala.runtime.AbstractPartialFunction.apply(AbstractPartialFunction.scala:38)
	at org.apache.spark.sql.catalyst.util.Iso8601TimestampFormatter.$anonfun$parse$1(TimestampFormatter.scala:86)
	at scala.runtime.java8.JFunction0$mcJ$sp.apply(JFunction0$mcJ$sp.java:23)
	at scala.Option.getOrElse(Option.scala:189)
	at org.apache.spark.sql.catalyst.util.Iso8601TimestampFormatter.parse(TimestampFormatter.scala:77)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.agg_doAggregateWithKeys_0$(Unknown Source)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.java:43)
	at org.apache.spark.sql.execution.WholeStageCodegenExec$$anon$1.hasNext(WholeStageCodegenExec.scala:757)
	at scala.collection.Iterator$$anon$10.hasNext(Iterator.scala:458)
	at org.apache.spark.shuffle.sort.BypassMergeSortShuffleWriter.write(BypassMergeSortShuffleWriter.java:132)
	at org.apache.spark.shuffle.ShuffleWriteProcessor.write(ShuffleWriteProcessor.scala:59)
	at org.apache.spark.scheduler.ShuffleMapTask.$anonfun$runTask$3(ShuffleMapTask.scala:81)
	at com.databricks.spark.util.ExecutorFrameProfiler$.record(ExecutorFrameProfiler.scala:110)
	at org.apache.spark.scheduler.ShuffleMapTask.$anonfun$runTask$1(ShuffleMapTask.scala:81)
	at com.databricks.spark.util.ExecutorFrameProfiler$.record(ExecutorFrameProfiler.scala:110)
	at org.apache.spark.scheduler.ShuffleMapTask.runTask(ShuffleMapTask.scala:

위의 코드 실행 결과 null 값을 가진 로우에서 전체 날짜의 합계 확인 가능. 롤업된 두 개의 컬럼값이 모두 null인 로우는 두 컬럼에 속한 레코드의 전체 합계를 나타냄.

In [0]:
rolledUpDF.where("Country IS NULL").show()

--------------------------------------------------------------------------- 
 Py4JJavaError Traceback (most recent call last)
 <command-1049518272702053> in <module> 
 ----> 1 rolledUpDF . where ( "Country IS NULL" ) . show ( ) 

 /databricks/spark/python/pyspark/sql/dataframe.py in show (self, n, truncate, vertical) 
 488 """
 489 if isinstance ( truncate , bool ) and truncate : 
 --> 490 print ( self . _jdf . showString ( n , 20 , vertical ) ) 
 491 else : 
 492 print ( self . _jdf . showString ( n , int ( truncate ) , vertical ) ) 

 /databricks/spark/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1302 
 1303 answer = self . gateway_client . send_command ( command ) 
 -> 1304 return_value = get_return_value(
 1305 answer, self.gateway_client, self.target_id, self.name)
 1306 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 115 def deco ( * a , ** kw ) : 
 116 try : 
 --> 117 return f ( * a , ** kw ) 
 118 except py4j . protocol . Py4JJavaError as e : 
 119 converted = convert_exception ( e . java_exception ) 

 /databricks/spark/python/lib/py4j-0.10.9-src.zip/py4j/protocol.py in get_return_value (answer, gateway_client, target_id, name) 
 324 value = OUTPUT_CONVERTER [ type ] ( answer [ 2 : ] , gateway_client ) 
 325 if answer [ 1 ] == REFERENCE_TYPE : 
 --> 326 raise Py4JJavaError(
 327 "An error occurred while calling {0}{1}{2}.\n" . 
 328 format(target_id, ".", name), value)

 Py4JJavaError : An error occurred while calling o1676.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 1 in stage 64.0 failed 1 times, most recent failure: Lost task 1.0 in stage 64.0 (TID 140) (ip-10-172-246-109.us-west-2.compute.internal executor driver): org.apache.spark.SparkUpgradeException: You may get a different result due to the upgrading of Spark 3.0: Fail to parse '5/13/2011 11:45' in the new parser. You can set spark.sql.legacy.timeParserPolicy to LEGACY to restore the behavior before Spark 3.0, or set to CORRECTED and treat it as an invalid datetime string.
	at org.apache.spark.sql.catalyst.util.DateTimeFormatterHelper$$anonfun$checkParsedDiff$1.applyOrElse(DateTimeFormatterHelper.scala:150)
	at org.apache.spark.sql.catalyst.util.DateTimeFormatterHelper$$anonfun$checkParsedDiff$1.applyOrElse(DateTimeFormatterHelper.scala:141)
	at scala.runtime.AbstractPartialFunction.apply(AbstractPartialFunction.scala:38)
	at org.apache.spark.sql.catalyst.util.Iso8601TimestampFormatter.$anonfun$parse$1(TimestampFormatter.scala:86)
	at scala.runtime.java8.JFunction0$mcJ$sp.apply(JFunction0$mcJ$sp.java:23)
	at scala.Option.getOrElse(Option.scala:189)
	at org.apache.spark.sql.catalyst.util.Iso8601TimestampFormatter.parse(TimestampFormatter.scala:77)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.agg_doAggregateWithKeys_0$(Unknown Source)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.java:43)
	at org.apache.spark.sql.execution.WholeStageCodegenExec$$anon$1.hasNext(WholeStageCodegenExec.scala:757)
	at scala.collection.Iterator$$anon$10.hasNext(Iterator.scala:458)
	at org.apache.spark.shuffle.sort.BypassMergeSortShuffleWriter.write(BypassMergeSortShuffleWriter.java:132)
	at org.apache.spark.shuffle.ShuffleWriteProcessor.write(ShuffleWriteProcessor.scala:59)
	at org.apache.spark.scheduler.ShuffleMapTask.$anonfun$runTask$3(ShuffleMapTask.scala:81)
	at com.databricks.spark.util.ExecutorFrameProfiler$.record(ExecutorFrameProfiler.scala:110)
	at org.apache.spark.scheduler.ShuffleMapTask.$anonfun$runTask$1(ShuffleMapTask.scala:81)
	at com.databricks.spark.util.ExecutorFrameProfiler$.record(ExecutorFrameProfiler.scala:110)
	at org.apache.spark.scheduler.ShuffleMapTask.runTask(ShuffleMapTask.scala:53)
	at org.apache.spark.scheduler.ShuffleMapTask.runTask(ShuffleMapTask.sc

In [0]:
rolledUpDF.where("Date IS NULL").show()

--------------------------------------------------------------------------- 
 Py4JJavaError Traceback (most recent call last)
 <command-1049518272702054> in <module> 
 ----> 1 rolledUpDF . where ( "Date IS NULL" ) . show ( ) 

 /databricks/spark/python/pyspark/sql/dataframe.py in show (self, n, truncate, vertical) 
 488 """
 489 if isinstance ( truncate , bool ) and truncate : 
 --> 490 print ( self . _jdf . showString ( n , 20 , vertical ) ) 
 491 else : 
 492 print ( self . _jdf . showString ( n , int ( truncate ) , vertical ) ) 

 /databricks/spark/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1302 
 1303 answer = self . gateway_client . send_command ( command ) 
 -> 1304 return_value = get_return_value(
 1305 answer, self.gateway_client, self.target_id, self.name)
 1306 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 115 def deco ( * a , ** kw ) : 
 116 try : 
 --> 117 return f ( * a , ** kw ) 
 118 except py4j . protocol . Py4JJavaError as e : 
 119 converted = convert_exception ( e . java_exception ) 

 /databricks/spark/python/lib/py4j-0.10.9-src.zip/py4j/protocol.py in get_return_value (answer, gateway_client, target_id, name) 
 324 value = OUTPUT_CONVERTER [ type ] ( answer [ 2 : ] , gateway_client ) 
 325 if answer [ 1 ] == REFERENCE_TYPE : 
 --> 326 raise Py4JJavaError(
 327 "An error occurred while calling {0}{1}{2}.\n" . 
 328 format(target_id, ".", name), value)

 Py4JJavaError : An error occurred while calling o1696.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 2 in stage 65.0 failed 1 times, most recent failure: Lost task 2.0 in stage 65.0 (TID 146) (ip-10-172-246-109.us-west-2.compute.internal executor driver): org.apache.spark.SparkUpgradeException: You may get a different result due to the upgrading of Spark 3.0: Fail to parse '5/13/2011 11:45' in the new parser. You can set spark.sql.legacy.timeParserPolicy to LEGACY to restore the behavior before Spark 3.0, or set to CORRECTED and treat it as an invalid datetime string.
	at org.apache.spark.sql.catalyst.util.DateTimeFormatterHelper$$anonfun$checkParsedDiff$1.applyOrElse(DateTimeFormatterHelper.scala:150)
	at org.apache.spark.sql.catalyst.util.DateTimeFormatterHelper$$anonfun$checkParsedDiff$1.applyOrElse(DateTimeFormatterHelper.scala:141)
	at scala.runtime.AbstractPartialFunction.apply(AbstractPartialFunction.scala:38)
	at org.apache.spark.sql.catalyst.util.Iso8601TimestampFormatter.$anonfun$parse$1(TimestampFormatter.scala:86)
	at scala.runtime.java8.JFunction0$mcJ$sp.apply(JFunction0$mcJ$sp.java:23)
	at scala.Option.getOrElse(Option.scala:189)
	at org.apache.spark.sql.catalyst.util.Iso8601TimestampFormatter.parse(TimestampFormatter.scala:77)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.agg_doAggregateWithKeys_0$(Unknown Source)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.java:43)
	at org.apache.spark.sql.execution.WholeStageCodegenExec$$anon$1.hasNext(WholeStageCodegenExec.scala:757)
	at scala.collection.Iterator$$anon$10.hasNext(Iterator.scala:458)
	at org.apache.spark.shuffle.sort.BypassMergeSortShuffleWriter.write(BypassMergeSortShuffleWriter.java:132)
	at org.apache.spark.shuffle.ShuffleWriteProcessor.write(ShuffleWriteProcessor.scala:59)
	at org.apache.spark.scheduler.ShuffleMapTask.$anonfun$runTask$3(ShuffleMapTask.scala:81)
	at com.databricks.spark.util.ExecutorFrameProfiler$.record(ExecutorFrameProfiler.scala:110)
	at org.apache.spark.scheduler.ShuffleMapTask.$anonfun$runTask$1(ShuffleMapTask.scala:81)
	at com.databricks.spark.util.ExecutorFrameProfiler$.record(ExecutorFrameProfiler.scala:110)
	at org.apache.spark.scheduler.ShuffleMapTask.runTask(ShuffleMapTask.scala:53)
	at org.apache.spark.scheduler.ShuffleMapTask.runTask(ShuffleMapTask.scala

### 7.4.2 큐브

큐브는 롤업을 고차원적으로 사용할 수 있게 해준다. - 큐브는 요소들을 계층적으로 다루는 대신 모든 차원에 대해 동일한 작업 수행. 

즉, 전체 기간에 대해 날짜와 국가별 결과를 얻을 수 있다.

코드) 다음의 정보를 가진 테이블 생성

* 전체 날짜와 모든 국가에 대한 집계
* 모든 국가의 날짜별 집계
* 날짜별 국가별 합계
* 전체 날짜의 국가별 합계

메서드 호출 방식은 롤업과 매우 유사하며 rollup 메서드 대신 cube 메서드 호출

In [0]:
from pyspark.sql.functions import sum

dfNoNull.cube("Date", "Country").agg(sum(col("Quantity")))\
  .select("Date", "Country", "sum(Quantity)").orderBy("Date").show()

--------------------------------------------------------------------------- 
 Py4JJavaError Traceback (most recent call last)
 <command-1049518272702056> in <module> 
 1 from pyspark . sql . functions import sum
 2 
 ----> 3 dfNoNull . cube ( "Date" , "Country" ) . agg ( sum ( col ( "Quantity" ) ) ) \ 
 4 . select ( "Date" , "Country" , "sum(Quantity)" ) . orderBy ( "Date" ) . show ( ) 

 /databricks/spark/python/pyspark/sql/dataframe.py in show (self, n, truncate, vertical) 
 488 """
 489 if isinstance ( truncate , bool ) and truncate : 
 --> 490 print ( self . _jdf . showString ( n , 20 , vertical ) ) 
 491 else : 
 492 print ( self . _jdf . showString ( n , int ( truncate ) , vertical ) ) 

 /databricks/spark/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1302 
 1303 answer = self . gateway_client . send_command ( command ) 
 -> 1304 return_value = get_return_value(
 1305 answer, self.gateway_client, self.target_id, self.name)
 1306 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 115 def deco ( * a , ** kw ) : 
 116 try : 
 --> 117 return f ( * a , ** kw ) 
 118 except py4j . protocol . Py4JJavaError as e : 
 119 converted = convert_exception ( e . java_exception ) 

 /databricks/spark/python/lib/py4j-0.10.9-src.zip/py4j/protocol.py in get_return_value (answer, gateway_client, target_id, name) 
 324 value = OUTPUT_CONVERTER [ type ] ( answer [ 2 : ] , gateway_client ) 
 325 if answer [ 1 ] == REFERENCE_TYPE : 
 --> 326 raise Py4JJavaError(
 327 "An error occurred while calling {0}{1}{2}.\n" . 
 328 format(target_id, ".", name), value)

 Py4JJavaError : An error occurred while calling o1735.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 2 in stage 66.0 failed 1 times, most recent failure: Lost task 2.0 in stage 66.0 (TID 151) (ip-10-172-246-109.us-west-2.compute.internal executor driver): org.apache.spark.SparkUpgradeException: You may get a different result due to the upgrading of Spark 3.0: Fail to parse '5/13/2011 11:45' in the new parser. You can set spark.sql.legacy.timeParserPolicy to LEGACY to restore the behavior before Spark 3.0, or set to CORRECTED and treat it as an invalid datetime string.
	at org.apache.spark.sql.catalyst.util.DateTimeFormatterHelper$$anonfun$checkParsedDiff$1.applyOrElse(DateTimeFormatterHelper.scala:150)
	at org.apache.spark.sql.catalyst.util.DateTimeFormatterHelper$$anonfun$checkParsedDiff$1.applyOrElse(DateTimeFormatterHelper.scala:141)
	at scala.runtime.AbstractPartialFunction.apply(AbstractPartialFunction.scala:38)
	at org.apache.spark.sql.catalyst.util.Iso8601TimestampFormatter.$anonfun$parse$1(TimestampFormatter.scala:86)
	at scala.runtime.java8.JFunction0$mcJ$sp.apply(JFunction0$mcJ$sp.java:23)
	at scala.Option.getOrElse(Option.scala:189)
	at org.apache.spark.sql.catalyst.util.Iso8601TimestampFormatter.parse(TimestampFormatter.scala:77)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.agg_doAggregateWithKeys_0$(Unknown Source)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.java:43)
	at org.apache.spark.sql.execution.WholeStageCodegenExec$$anon$1.hasNext(WholeStageCodegenExec.scala:757)
	at scala.collection.Iterator$$anon$10.hasNext(Iterator.scala:458)
	at org.apache.spark.shuffle.sort.BypassMergeSortShuffleWriter.write(BypassMergeSortShuffleWriter.java:132)
	at org.apache.spark.shuffle.ShuffleWriteProcessor.write(ShuffleWriteProcessor.scala:59)
	at org.apache.spark.scheduler.ShuffleMapTask.$anonfun$runTask$3(ShuffleMapTask.scala:81)
	at com.databricks.spark.util.ExecutorFrameProfiler$.record(ExecutorFrameProfiler.scala:110)
	at org.apache.spark.scheduler.ShuffleMapTask.$anonfun$runTask$1(ShuffleMapTask.scala:81)
	at com.databricks.spark.util.ExecutorFrameProfiler$.record(ExecutorFrameProfiler.

큐브를 사용하면 테이블에 있는 모든 정보를 빠르고 쉽게 조회할 수 잇는 요약 정보 테이블을 만들 수 있다.

### 7.4.3 그룹화 메타데이터

큐브와 롤업을 사용하다 보면 집계 수준에 따라 쉽게 필터링하기 위해 집계 수준을 조회하는 경우가 발생하는데 이때 grouping_id를 사용. 

grouping_id는 결과 데이터셋의 집계 수준을 명시하는 컬럼을 제공. 

< 그룹화 ID의 의미 >
* 3 : 가장 높은 계층의 집계 결과에서 나타나며 customerId나 stockCode에 관계없이 총 수량 제공.
* 2 : 개별 재고 코드의 모든 집계 결과에서 나타나며 customerId에 관계없이 재고 코드별 총 수량 제공.
* 1 : 구매한 물품에 관계없이 customerId를 기반으로 총 수량 제공.
* 0 : customerId나 stockCode별 조합에 따라 총 수량 제공.

### 7.4.4 피벗

피벗을 사용해 로우를 컬럼으로 변환 가능. 현재 데이터셋에는 Country 컬럼이 있다. 피벗을 사용해 국가별로 집계 함수를 적용할 수 있고 쿼리를 사용해 쉽게 결과 확인 가능.

In [0]:
pivoted = dfWithDate.groupBy("date").pivot("Country").sum()

DataFrame은 국가명, 수치형 변수 그리고 날짜를 나타내는 컬럼을 조합한 컬럼을 가진다. 

ex) USA와 관련된 컬럼을 살펴보면 USA_sum(Quantity), USA_sum(UnitPrice) 그리고 USA_sum(CustomerID)가 있다. 

또 집계를 수행했기 때문에 수치형 컬럼으로 나타난다.

In [0]:
pivoted.where("date > '2011-12-05'").select("date", "'USA_sum(Quantity)'").show()

--------------------------------------------------------------------------- 
 AnalysisException Traceback (most recent call last)
 <command-1049518272702062> in <module> 
 ----> 1 pivoted . where ( "date > '2011-12-05'" ) . select ( "date" , "'USA_sum(Quantity)'" ) . show ( ) 

 /databricks/spark/python/pyspark/sql/dataframe.py in select (self, *cols) 
 1690 [ Row ( name = 'Alice' , age = 12 ) , Row ( name = 'Bob' , age = 15 ) ] 
 1691 """
 -> 1692 jdf = self . _jdf . select ( self . _jcols ( * cols ) ) 
 1693 return DataFrame ( jdf , self . sql_ctx ) 
 1694 

 /databricks/spark/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1302 
 1303 answer = self . gateway_client . send_command ( command ) 
 -> 1304 return_value = get_return_value(
 1305 answer, self.gateway_client, self.target_id, self.name)
 1306 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 121 # Hide where the exception came from that shows a non-Pythonic 
 122 # JVM exception message. 
 --> 123 raise converted from None 
 124 else : 
 125 raise 

 AnalysisException : cannot resolve '`'USA_sum(Quantity)'`' given input columns: [Australia_sum(CustomerID), Australia_sum(Quantity), Australia_sum(UnitPrice), Austria_sum(CustomerID), Austria_sum(Quantity), Austria_sum(UnitPrice), Bahrain_sum(CustomerID), Bahrain_sum(Quantity), Bahrain_sum(UnitPrice), Belgium_sum(CustomerID), Belgium_sum(Quantity), Belgium_sum(UnitPrice), Brazil_sum(CustomerID), Brazil_sum(Quantity), Brazil_sum(UnitPrice), Canada_sum(CustomerID), Canada_sum(Quantity), Canada_sum(UnitPrice), Channel Islands_sum(CustomerID), Channel Islands_sum(Quantity), Channel Islands_sum(UnitPrice), Cyprus_sum(CustomerID), Cyprus_sum(Quantity), Cyprus_sum(UnitPrice), Czech Republic_sum(CustomerID), Czech Republic_sum(Quantity), Czech Republic_sum(UnitPrice), Denmark_sum(CustomerID), Denmark_sum(Quantity), Denmark_sum(UnitPrice), EIRE_sum(CustomerID), EIRE_sum(Quantity), EIRE_sum(UnitPrice), European Community_sum(CustomerID), European Community_sum(Quantity), European Community_sum(UnitPrice), Finland_sum(CustomerID), Finland_sum(Quantity), Finland_sum(UnitPrice), France_sum(CustomerID), France_sum(Quantity), France_sum(UnitPrice), Germany_sum(CustomerID), Germany_sum(Quantity), Germany_sum(UnitPrice), Greece_sum(CustomerID), Greece_sum(Quantity), Greece_sum(UnitPrice), Hong Kong_sum(CustomerID), Hong Kong_sum(Quantity), Hong Kong_sum(UnitPrice), Iceland_sum(CustomerID), Iceland_sum(Quantity), Iceland_sum(UnitPrice), Israel_sum(CustomerID), Israel_sum(Quantity), Israel_sum(UnitPrice), Italy_sum(CustomerID), Italy_sum(Quantity), Italy_sum(UnitPrice), Japan_sum(CustomerID), Japan_sum(Quantity), Japan_sum(UnitPrice), Lebanon_sum(CustomerID), Lebanon_sum(Quantity), Lebanon_sum(UnitPrice), Lithuania_sum(CustomerID), Lithuania_sum(Quantity), Lithuania_sum(UnitPrice), Malta_sum(CustomerID), Malta_sum(Quantity), Malta_sum(UnitPrice), Netherlands_sum(CustomerID), Netherlands_sum(Quantity), Netherlands_sum(UnitPrice), Norway_sum(CustomerID), Norway_sum(Quantity), Norway_sum(UnitPrice), Poland_sum(CustomerID), Poland_sum(Quantity), Poland_sum(UnitPrice), Portugal_sum(CustomerID), Portugal_sum(Quantity), Portugal_sum(UnitPrice), RSA_sum(CustomerID), RSA_sum(Quantity), RSA_sum(UnitPrice), Saudi Arabia_sum(CustomerID), Saudi Arabia_sum(Quantity), Saudi Arabia_sum(UnitPrice), Singapore_sum(CustomerID), Singapore_sum(Quantity), Singapore_sum(UnitPrice), Spain_sum(CustomerID), Spain_sum(Quantity), Spain_sum(UnitPrice), Sweden_sum(CustomerID), Sweden_sum(Quantity), Sweden_sum(UnitPrice), Switzerland_sum(CustomerID), Switzerland_sum(Quantity), Switzerland_sum(UnitPrice), USA_sum(CustomerID), USA_sum(Quantity), USA_sum(UnitPrice), United Arab Emirates_sum(CustomerID), United Arab Emirates_sum(Quantity), United Arab Emirates_sum(UnitPrice), United Kingdom_sum(CustomerID), United Kingdom_sum(Quantity), United Kingdom_sum(UnitPrice), Unspecified_sum(CustomerID), Unspecified_sum(Quantity), 

데이터를 탐색하는 방식에 따라 피벗을 수행한 결과값이 감소할 수 있다. 

따라서 카디널리티가 낮다면 스키마와 쿼리 대상을 확인할 수 있도록 피벗을 사용해 다수의 컬럼으로 변환하는 것이 좋다.

## 7.5 사용자 정의 집계 함수

사용자 정의 집계 함수는 직접 제작한 함수나 비지니스 규칙에 기반을 둔 자체 집계 함수를 정의하는 방법. 

UDAF를 사용해 입력 데이터 그룹에 직접 개발한 연산 수행 가능. 

스파크는 입력 데이터의 모든 그룹의 중간 결과를 단일 AggregationBuffer에 저장해 관리. 

UDAF를 생성하려면 기본 클래스인 UserDefinedAggregateFunction을 상속받아 다음과 같은 메서드 정의
* inputShema : UDAF 입력 파라미터의 스키마를 StructType으로 정의
* bufferSchema : UDAF 중간 결과의 스키마를 StructType으로 정의
* dataType : 반환될 값의 DataType 정의
* deterministic : UDAF가 동일한 입력값에 대해 항상 동일한 결과를 반환하는지 불리언 값으로 정의
* initialize : 집계용 버퍼의 값을 초기화하는 로직 정의
* update : 입력받은 로우를 기반으로 내부 버퍼를 업데이트하는 로직을 정의
* merge : 두 개의 집계용 버퍼를 병합하는 로직 정의
* evaluate : 집계의 최종 결과를 생성하는 로직 정의


UDAF는 현재 스칼라와 자바로만 사용할 수 있다.